In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
# Base model construction
model_comps = ["susceptible", "early latent", "late latent", "infectious", "recovered"]
model = CompartmentalModel(
    times=[1850.0, 2024.0],
    compartments=model_comps,
    infectious_compartments=["infectious"],
)
model.set_initial_population({"susceptible": 500.0, "infectious": 500.0})

In [ ]:
# TB transitions
model.add_death_flow("TB death", Parameter("death rate"), "infectious")
model.add_transition_flow("silly_transition", Parameter("silly rate"), "susceptible", "infectious")

In [ ]:
# Demographic transitions
model.add_universal_death_flows("population_death", Parameter("population death rate"))
model.add_replacement_birth_flow("replacement_birth", "susceptible")

In [ ]:
from summer2 import flows
from summer2.model import _validate_flowparam
from summer2.adjust import FlowParam
from typing import Dict, Optional


In [ ]:
def add_crude_birth_flow(
    model,
    name: str,
    birth_rate: FlowParam,
    dest: str,
    dest_strata: Optional[Dict[str, str]] = None,
    expected_flow_count: Optional[int] = None,
):
    _validate_flowparam(birth_rate)
    model._add_entry_flow(
        flows.CrudeBirthFlow,
        name,
        birth_rate,
        dest,
        dest_strata,
        expected_flow_count,
    )

In [ ]:
add_crude_birth_flow(model, "extra_birth", Parameter("extra birth"), "susceptible")

In [ ]:
model.request_output_for_compartments("total_population", model_comps);

In [ ]:
params = {
    "silly rate": 1.0,
    "extra birth": 0.001,
    "death rate": 0.1,
    "population death rate": 0.01,
}

In [ ]:
model.run(params)

In [ ]:
model.get_outputs_df().plot.area()

In [ ]:
model.get_derived_outputs_df()